# **COBEQ 2025**

## Libs

In [1]:
import pandas as pd
import numpy as np
np.random.seed(1)
import optuna
import itertools
import shutil
from functools import partial

import plotly.graph_objects as go
import plotly.express as px
import plotly.subplots
import time

from neuralforecast import NeuralForecast
from neuralforecast.models import NBEATS, NHITS
import sklearn.metrics as metrics
import scipy.stats as stats

ponte = pd.read_pickle(r'Data\Data_Ponte_dos_Remedios.pkl')
del ponte['o3']
guarulhos = pd.read_pickle(r'Data\Data_Guarulhos.pkl')
guarulhos = guarulhos[['date','o3']]

data = ponte.merge(guarulhos, on='date', how='outer')
data.reset_index(drop=True)

import joblib
import pickle
from IPython.display import clear_output
import os
os.environ['NIXTLA_ID_AS_COL'] = '1'

from pytorch_lightning import Trainer
trainer = Trainer(
    max_steps=4,
    logger=False,
    enable_progress_bar=False,
    enable_model_summary=False  # Disable model summary
)

import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="optuna")

from TimeObjectModule import TimeObject

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


## **Experimental Planning**

In [75]:
for h in [14, 30, 60, 90, 120]:
    for k in [1, 2, 3, 4, 5,]:
        x = TimeObject(data, column='pm10', agg_freq='D')
        x.fixed_origin_rolling_window_cross_validation(
            initial_train_date=0,
            total_data_points=1000+(k*h),
            h=h,
            # plot_interval=True,
            width=1100,height=400
        )
        # print()

In [79]:
for horizon in [14, 30, 60, 90, 120, 180]:
    print(f'H = {horizon}')
    max_k_fold = 3
    for k_fold in range(0,max_k_fold):
        obj = TimeObject(df=data, column='pm10', agg_freq='D')
        obj.fixed_origin_rolling_window_cross_validation(
            initial_train_date=None,
            total_data_points=(len(obj.nixtla_df))-(k_fold*horizon),
            h=horizon,
            # plot_interval=True
        )
        print(f'Fold: {k_fold+1} | Train = {len(obj.Y_train)} | Percent = {round(100*len(obj.Y_train)/len(obj.nixtla_df),3)}%')
    print('=================')
    for k_fold in range(0,max_k_fold):
        obj = TimeObject(df=data, column='pm10', agg_freq='D')
        obj.fixed_origin_rolling_window_cross_validation(
            initial_train_date=None,
            total_data_points=(len(obj.nixtla_df)-(max_k_fold*horizon))-(k_fold*horizon),
            h=horizon,
            # plot_interval=True
        )
        print(f'Fold: {k_fold+1} | Train = {len(obj.Y_train)} | Percent = {round(100*len(obj.Y_train)/len(obj.nixtla_df),3)}%')

    print('\n')

H = 14
Fold: 1 | Train = 2452 | Percent = 99.432%
Fold: 2 | Train = 2438 | Percent = 98.865%
Fold: 3 | Train = 2424 | Percent = 98.297%
Fold: 1 | Train = 2410 | Percent = 97.729%
Fold: 2 | Train = 2396 | Percent = 97.161%
Fold: 3 | Train = 2382 | Percent = 96.594%


H = 30
Fold: 1 | Train = 2436 | Percent = 98.783%
Fold: 2 | Train = 2406 | Percent = 97.567%
Fold: 3 | Train = 2376 | Percent = 96.35%
Fold: 1 | Train = 2346 | Percent = 95.134%
Fold: 2 | Train = 2316 | Percent = 93.917%
Fold: 3 | Train = 2286 | Percent = 92.701%


H = 60
Fold: 1 | Train = 2406 | Percent = 97.567%
Fold: 2 | Train = 2346 | Percent = 95.134%
Fold: 3 | Train = 2286 | Percent = 92.701%
Fold: 1 | Train = 2226 | Percent = 90.268%
Fold: 2 | Train = 2166 | Percent = 87.835%
Fold: 3 | Train = 2106 | Percent = 85.401%


H = 90
Fold: 1 | Train = 2376 | Percent = 96.35%
Fold: 2 | Train = 2286 | Percent = 92.701%
Fold: 3 | Train = 2196 | Percent = 89.051%
Fold: 1 | Train = 2106 | Percent = 85.401%
Fold: 2 | Train = 2016

In [82]:
max_k_fold = 4
horizon = 30
k_fold = 4

obj = TimeObject(df=data, column='pm10', agg_freq='D')
obj.fixed_origin_rolling_window_cross_validation(
    initial_train_date=None,
    total_data_points=(len(obj.nixtla_df)-(max_k_fold*horizon))-(k_fold*horizon),
    h=horizon,
    plot_interval=True
);
print(f'{round(100*len(obj.Y_train)/len(obj.nixtla_df),3)}')

k_fold = 1
obj = TimeObject(df=data, column='pm10', agg_freq='D')
obj.fixed_origin_rolling_window_cross_validation(
    initial_train_date=None,
    total_data_points=(len(obj.nixtla_df))-(k_fold*horizon),
    h=horizon,
    plot_interval=True
);
print(f'{round(100*len(obj.Y_train)/len(obj.nixtla_df),3)}')

89.051


97.567


## **Optuna**

### N-HiTS

In [ ]:
# Define the objective function
def objective(trial, pollutant, horizon):
    # Hyperparameter search space
    input_size = trial.suggest_int('input_size', 90, 1100, step=1)
    n_stacks = trial.suggest_int('n_stacks', 3, 7, step=1)
    n_blocks = trial.suggest_int('n_blocks', 1, 7, step=1)
    max_steps = trial.suggest_int('max_steps', 10, 700, step=1)
    local_scalar_type = trial.suggest_categorical('local_scalar_type', [None, 'standard', 'boxcox', 'minmax'])
    n_pool_kernel_size = trial.suggest_categorical('n_pool_kernel_size', [list(combination) for combination in list(itertools.product([1, 2, 3], repeat=3))])
    n_freq_downsample = trial.suggest_categorical('n_freq_downsample', [list(combination) for combination in list(itertools.product([1, 7, 90, 180, 365], repeat=3))])

    mape = []
    smape = []
    max = []
    mae = []
    mse = []
    # Split for cross validation
    max_k_fold = 4
    for k_fold in range(0,max_k_fold):
        print(f'\nPollutant = {pollutant} \nh = {horizon} \nTrial = {trial.number+1}\nFold = {k_fold+1}\n')
        # Instantiate TimeObject and prepare training data
        obj = TimeObject(df=data, column=pollutant, agg_freq='D')
        obj.fixed_origin_rolling_window_cross_validation(
            initial_train_date=None,
            total_data_points=(len(obj.nixtla_df)-(max_k_fold*horizon))-(k_fold*horizon),
            h=horizon,
            plot_interval=False
        )

        # Define the model
        model = NHITS(
            h=horizon,
            input_size=input_size,
            stack_types=n_stacks*['identity'],
            n_freq_downsample=n_freq_downsample+(n_stacks-len(n_freq_downsample))*[1],
            n_blocks=n_stacks*[n_blocks],
            n_pool_kernel_size=(n_stacks-len(n_pool_kernel_size))*[1]+n_pool_kernel_size,
            pooling_mode="MaxPool1d",
            activation="ReLU",
            interpolation_mode='linear',
            max_steps=max_steps,
            val_check_steps=10,
            early_stop_patience_steps=int(np.round(max_steps/(20),0)),
        )

        # Initialize NeuralForecast and fit the model
        fcst = NeuralForecast(
            models=[model],
            freq='D',
            local_scaler_type=local_scalar_type
        )
        fcst.fit(df=obj.Y_train, verbose=False, val_size=horizon+1)
        prediction = fcst.predict(df=obj.Y_train, verbose=False)

        # Evaluate metrics
        obj.metrics_(forecast_df=prediction, method='NHITS')
        mape.append(obj.metrics['mape'])
        smape.append(obj.metrics['smape'])
        max.append(obj.metrics['max'])
        mae.append(obj.metrics['mae'])
        mse.append(obj.metrics['mse'])
        
        clear_output(wait=True)

    try:
        directory_path = "lightning_logs"
        if os.path.exists(directory_path):
            shutil.rmtree(directory_path)
    except:
        ...

    mape = np.mean(mape)
    smape = np.mean(smape)
    max = np.mean(max)
    mae = np.mean(mae)
    mse = np.mean(mse)

    # Collect the results
    results.append({
        'pollutant': pollutant,
        'freq': 'D',
        'fold': max_k_fold,
        'trial': trial.number+1,
        'train_len': f'{round(100*len(obj.Y_train)/len(obj.nixtla_df),3)}',
        'h': horizon,
        'input_size': input_size,
        'n_stacks': n_stacks,
        'n_blocks': n_blocks,
        'max_steps': max_steps,
        'local_scalar_type': local_scalar_type,
        'n_pool_kernel_size': n_pool_kernel_size,
        'n_freq_downsample': n_freq_downsample,
        'mape': mape,
        'smape': smape,
        'max': max,
        'mae': mae,
        'mse': mse,
    })

    # The objective for Optuna is to minimize the MAE (or maximize a metric)
    return smape, mae  # Any metric you want to optimize

for pollutant in data[['pm10']]:
    for h in [30, 60, 90]:
        # Initialize the results list
        results = []
        # Define the optimization study_nhits
        study_nhits = optuna.create_study(directions=['minimize','minimize'])  # Minimize the MAE

        # Run the optimization with the number of trials you want
        study_nhits.optimize(partial(objective, pollutant=pollutant, horizon=h), n_trials=200)

        clear_output(wait=True)
        NHITS_W = pd.DataFrame(results)

        output_dir = fr'Results COBEQ\NHITS (D)\{pollutant}'
        os.makedirs(output_dir, exist_ok=True)
        NHITS_W.to_pickle(fr'Results COBEQ\NHITS (D)\{pollutant}\{h}D_Df.pkl')
        joblib.dump(study_nhits, fr"Results COBEQ\NHITS (D)\{pollutant}\{h}D_Study.pkl")

### NBEATS

In [ ]:
# Define the objective function
def objective(trial, pollutant, horizon):
    # Hyperparameter search space
    input_size = trial.suggest_int('input_size', 90, 1100, step=1)
    n_stacks = trial.suggest_int('n_stacks', 2, 7, step=1)
    n_blocks = trial.suggest_int('n_blocks', 1, 5, step=1)
    max_steps = trial.suggest_int('max_steps', 10, 700, step=1)
    local_scalar_type = trial.suggest_categorical('local_scalar_type', [None, 'standard', 'boxcox', 'minmax'])
    interpretability = trial.suggest_categorical('interpretability', [list(combination) for combination in list(itertools.product(['seasonality', 'trend', 'identity'], repeat=2))])

    mape = []
    smape = []
    max = []
    mae = []
    mse = []
    # Split for cross validation
    max_k_fold = 4
    for k_fold in range(0,max_k_fold):
        print(f'\nPollutant = {pollutant} \nh = {horizon} \nTrial = {trial.number+1}\nFold = {k_fold+1}\n')
        # Instantiate TimeObject and prepare training data
        obj = TimeObject(df=data, column=pollutant, agg_freq='D')
        obj.fixed_origin_rolling_window_cross_validation(
            initial_train_date=None,
            total_data_points=(len(obj.nixtla_df)-(max_k_fold*horizon))-(k_fold*horizon),
            h=horizon,
            plot_interval=False
        )

        # Define the model
        model = NBEATS(
            h=horizon,
            input_size=input_size,
            stack_types=interpretability+(n_stacks-len(interpretability))*['identity'],
            n_blocks=n_stacks * [n_blocks],
            max_steps=max_steps,
            learning_rate=1e-3,
            val_check_steps=10,
            early_stop_patience_steps=int(np.round(max_steps/(20),0)),
        )

        # Initialize NeuralForecast and fit the model
        fcst = NeuralForecast(
            models=[model],
            freq='D',
            local_scaler_type=local_scalar_type
        )
        fcst.fit(df=obj.Y_train, verbose=False, val_size=horizon+1)
        prediction = fcst.predict(df=obj.Y_train, verbose=False)

        # Evaluate metrics
        obj.metrics_(forecast_df=prediction, method='NBEATS')
        mape.append(obj.metrics['mape'])
        smape.append(obj.metrics['smape'])
        max.append(obj.metrics['max'])
        mae.append(obj.metrics['mae'])
        mse.append(obj.metrics['mse'])
            
        clear_output(wait=True)

        try:
            directory_path = "lightning_logs"
            if os.path.exists(directory_path):
                shutil.rmtree(directory_path)
        except:
            ...

    mape = np.mean(mape)
    smape = np.mean(smape)
    max = np.mean(max)
    mae = np.mean(mae)
    mse = np.mean(mse)

    # Collect the results
    results.append({
        'pollutant': pollutant,
        'freq': 'D',
        'fold': max_k_fold,
        'trial': trial.number+1,
        'train_len': f'{round(100*len(obj.Y_train)/len(obj.nixtla_df),3)}',
        'h': horizon,
        'input_size': input_size,
        'n_stacks': n_stacks,
        'n_blocks': n_blocks,
        'max_steps': max_steps,
        'local_scalar_type': local_scalar_type,
        'interpretability': interpretability,
        'mape': mape,
        'smape': smape,
        'max': max,
        'mae': mae,
        'mse': mse,
    })

    # The objective for Optuna is to minimize the MAE (or maximize a metric)
    return smape, mae  # Any metric you want to optimize

for pollutant in data[['pm10']]:
    for h in [30, 60, 90]:
        # Initialize the results list
        results = []
        # Define the optimization study_nbeats
        study_nbeats = optuna.create_study(directions=['minimize','minimize'])  # Minimize the MAE

        # Run the optimization with the number of trials you want
        study_nbeats.optimize(partial(objective, pollutant=pollutant, horizon=h), n_trials=200)

        clear_output(wait=True)
        NBEATS_W = pd.DataFrame(results)

        output_dir = fr'Results COBEQ\NBEATS (D)\{pollutant}'
        os.makedirs(output_dir, exist_ok=True)
        NBEATS_W.to_pickle(fr'Results COBEQ\NBEATS (D)\{pollutant}\{h}D_Df.pkl')
        joblib.dump(study_nbeats, fr"Results COBEQ\NBEATS (D)\{pollutant}\{h}D_Study.pkl")

[I 2025-04-13 09:47:10,279] Trial 199 finished with values: [39.852276, 9.647514000000001] and parameters: {'input_size': 925, 'n_stacks': 2, 'n_blocks': 2, 'max_steps': 494, 'local_scalar_type': 'boxcox', 'interpretability': ['trend', 'identity']}.


## **Statistical**

In [ ]:
from statsforecast import StatsForecast
from statsforecast.models import AutoCES, AutoETS, AutoTheta

for pollutant in ['pm10',]:
    for h in [30, 60, 90]: #  30, 60, 90, 120
        results_stats = pd.DataFrame()

        mape = []
        smape = []
        max = []
        mae = []
        mse = []

        max_k_fold = 4
        for k_fold in range(0,max_k_fold):
            print(f'\nPollutant = {pollutant} \nh = {h}\nFold = {k_fold+1}\n')
            # Instantiate TimeObject and prepare training data
            obj = TimeObject(df=data, column=pollutant, agg_freq='D')
            obj.fixed_origin_rolling_window_cross_validation(
                initial_train_date=None,
                total_data_points=(len(obj.nixtla_df))-(k_fold*h),
                h=h,
                plot_interval=False
            )

            season_length = 365 # Monthly data 

            models = [
                # AutoARIMA(season_length=season_length, alias='AutoARIMA'),
                AutoCES(season_length=season_length, model='Z', alias='AutoCES-Z'),
                AutoCES(season_length=season_length, model='S', alias='AutoCES-S'),
                AutoCES(season_length=season_length, model='P', alias='AutoCES-P'),
                AutoCES(season_length=season_length, model='N', alias='AutoCES-N'),
                AutoTheta(season_length=season_length, decomposition_type="multiplicative", alias='AutoTheta-Multi'),
                AutoTheta(season_length=season_length, decomposition_type="additive", alias='AutoTheta-Add'),
            ]
            models = models + [
                AutoETS(season_length=season_length, model=ets, alias=f'AutoETS-{ets}')
                for ets in ['ZAZ', 'ZAN', 'ZAA', 'ZAM', 'ZNN']]

            frct = StatsForecast(models=models, freq='D')
            frct.fit(df=obj.Y_train)
            predicted = frct.predict(h=h)

            columns = predicted.columns
            columns = columns[(columns != 'ds') & (columns != 'unique_id')]

            for method in columns:
                obj.metrics_(predicted, method=method)
                results_stats = pd.concat([results_stats, pd.DataFrame({
                    'pollutant': [pollutant],
                    'method': [method],
                    'freq': ['D'],
                    'h': [h],
                    'mape': [obj.metrics['mape']],
                    'smape': [obj.metrics['smape']],
                    'max': [obj.metrics['max']],
                    'mae': [obj.metrics['mae']],
                    'mse': [obj.metrics['mse']]
                })])
            
            # ======================================================================================================

            nbeats = joblib.load(fr"Results COBEQ\NBEATS (D)\{pollutant}\{h}D_Study.pkl")
            model = NBEATS(
                h=h,
                input_size=nbeats.best_trials[0].params.get('input_size'),
                stack_types=nbeats.best_trials[0].params.get('interpretability')+(nbeats.best_trials[0].params.get('n_stacks')-len(nbeats.best_trials[0].params.get('interpretability')))*['identity'],
                n_blocks=nbeats.best_trials[0].params.get('n_stacks') * [nbeats.best_trials[0].params.get('n_blocks')],
                max_steps=nbeats.best_trials[0].params.get('max_steps'),
                learning_rate=1e-3,
                val_check_steps=10,
            )
            fcst = NeuralForecast(
                models=[model],
                freq='D',
                local_scaler_type=nbeats.best_trials[0].params.get('local_scalar_type')
            )
            fcst.fit(df=obj.Y_train, verbose=False)
            predicted = fcst.predict(df=obj.Y_train, verbose=False)
            obj.metrics_(predicted, method='NBEATS')
            results_stats = pd.concat([results_stats, pd.DataFrame({
                'pollutant': [pollutant],
                'method': ['NBEATS'],
                'freq': ['D'],
                'h': [h],
                'mape': [obj.metrics['mape']],
                'smape': [obj.metrics['smape']],
                'max': [obj.metrics['max']],
                'mae': [obj.metrics['mae']],
                'mse': [obj.metrics['mse']]
            })])

            # ======================================================================================================
            
            nhits = joblib.load(fr"Results COBEQ\NHITS (D)\{pollutant}\{h}D_Study.pkl")
            model = NHITS(
                h=h,
                input_size=nhits.best_trials[0].params.get('input_size'),
                stack_types=nhits.best_trials[0].params.get('n_stacks')*['identity'],
                n_freq_downsample=nhits.best_trials[0].params.get('n_freq_downsample')+(nhits.best_trials[0].params.get('n_stacks')-len(nhits.best_trials[0].params.get('n_freq_downsample')))*[1],
                n_blocks=nhits.best_trials[0].params.get('n_stacks')*[nhits.best_trials[0].params.get('n_blocks')],
                n_pool_kernel_size=(nhits.best_trials[0].params.get('n_stacks')-len(nhits.best_trials[0].params.get('n_pool_kernel_size')))*[1]+nhits.best_trials[0].params.get('n_pool_kernel_size'),
                pooling_mode="MaxPool1d",
                activation="ReLU",
                interpolation_mode='linear',
                max_steps=nhits.best_trials[0].params.get('max_steps'),
                val_check_steps=10,
            )
            fcst = NeuralForecast(
                models=[model],
                freq='D',
                local_scaler_type=nhits.best_trials[0].params.get('local_scalar_type')
            )
            fcst.fit(df=obj.Y_train, verbose=False)
            predicted = fcst.predict(df=obj.Y_train, verbose=False)
            obj.metrics_(predicted, method='NHITS')
            results_stats = pd.concat([results_stats, pd.DataFrame({
                'pollutant': [pollutant],
                'method': ['NHITS'],
                'freq': ['D'],
                'h': [h],
                'mape': [obj.metrics['mape']],
                'smape': [obj.metrics['smape']],
                'max': [obj.metrics['max']],
                'mae': [obj.metrics['mae']],
                'mse': [obj.metrics['mse']]
            })])
            
            clear_output(wait=True)

        # ======================================================================================================

        results_stats = pd.DataFrame(results_stats)

        output_dir = fr'Results COBEQ\Stats (D)\{pollutant}'
        os.makedirs(output_dir, exist_ok=True)
        results_stats.to_pickle(fr'Results COBEQ\Stats (D)\{pollutant}\{h}D_Df.pkl')

# **ANALYSIS**

#### Results Saved in an Excel File

In [ ]:
results_stats = joblib.load(r'Results COBEQ\Stats (D)\pm10\180D_Df.pkl')
results = results_stats.groupby(['pollutant','method','h','freq'])[['mape', 'smape', 'max', 'mae', 'mse']].mean().reset_index().sort_values('smape')
results

In [5]:
import joblib
import pandas as pd

# Load the dataset
results_30D = joblib.load(r'Results COBEQ\Stats (D)\pm10\30D_Df.pkl')
results_60D = joblib.load(r'Results COBEQ\Stats (D)\pm10\60D_Df.pkl')
results_90D = joblib.load(r'Results COBEQ\Stats (D)\pm10\90D_Df.pkl')

# Process the data for each dataset
def process_results(results_data):
    return results_data.groupby(['pollutant','method','h','freq'])[['mape', 'smape', 'max', 'mae', 'mse']].mean().reset_index().sort_values('smape')

# Get the results
results_30D_processed = process_results(results_30D)
results_60D_processed = process_results(results_60D)
results_90D_processed = process_results(results_90D)

# Create an Excel writer and save the results in different sheets
with pd.ExcelWriter(r'aggregated_results.xlsx') as writer:
    results_30D_processed.to_excel(writer, sheet_name='30D_Df', index=False)
    results_60D_processed.to_excel(writer, sheet_name='60D_Df', index=False)
    results_90D_processed.to_excel(writer, sheet_name='90D_Df', index=False)

"Excel file saved successfully."


'Excel file saved successfully.'

#### ANOVA Testing for Significance

In [6]:
results_stats = joblib.load(r'Results COBEQ\Stats (D)\pm10\30D_Df.pkl')
df = results_stats
df = df[['method','mape', 'smape', 'max', 'mae', 'mse']].reset_index(drop=True)

# Perform one-way ANOVA for each metric to check if NHITS is significantly different from others
metrics = ['mape', 'smape', 'max', 'mae', 'mse']
anova_results = {}

for metric in metrics:
    # Perform ANOVA
    groups = [df[df['method'] == method][metric] for method in df['method'].unique()]
    anova_results[metric] = stats.f_oneway(*groups)

anova_results

{'mape': F_onewayResult(statistic=np.float64(4.062932338687453), pvalue=np.float64(0.00041890162681600514)),
 'smape': F_onewayResult(statistic=np.float64(3.09825039920707), pvalue=np.float64(0.003699243361639936)),
 'max': F_onewayResult(statistic=np.float64(4.6083544878466745), pvalue=np.float64(0.0001316106903237089)),
 'mae': F_onewayResult(statistic=np.float64(3.589419630631214), pvalue=np.float64(0.001195826233925535)),
 'mse': F_onewayResult(statistic=np.float64(2.8933227585686847), pvalue=np.float64(0.0059920225689152114))}

In [36]:
from statsmodels.stats.multicomp import pairwise_tukeyhsd
long_df = pd.melt(df, id_vars=['method'], value_vars=['mape', 'smape', 'max', 'mae', 'mse'], 
                  var_name='metric', value_name='value')
tukey_results = {}

for metric in ['mape', 'smape', 'max', 'mae', 'mse']:
    print(f"Performing Tukey HSD for {metric}...")
    metric_data = long_df[long_df['metric'] == metric]
    tukey = pairwise_tukeyhsd(metric_data['value'], metric_data['method'])
    tukey_results[metric] = tukey
    print(tukey.summary())


Performing Tukey HSD for mape...
           Multiple Comparison of Means - Tukey HSD, FWER=0.05            
     group1          group2      meandiff p-adj    lower    upper   reject
--------------------------------------------------------------------------
      AutoCES-N       AutoCES-P   20.3953    1.0 -149.3712 190.1618  False
      AutoCES-N       AutoCES-S    3.4734    1.0 -166.2931 173.2399  False
      AutoCES-N       AutoCES-Z    6.1101    1.0 -163.6564 175.8766  False
      AutoCES-N     AutoETS-ZAA    4.6187    1.0 -165.1478 174.3852  False
      AutoCES-N     AutoETS-ZAM    7.9528    1.0 -161.8137 177.7194  False
      AutoCES-N     AutoETS-ZAN  194.7399  0.012   24.9734 364.5064   True
      AutoCES-N     AutoETS-ZAZ  194.0425 0.0125    24.276  363.809   True
      AutoCES-N     AutoETS-ZNN   21.1182    1.0 -148.6484 190.8847  False
      AutoCES-N   AutoTheta-Add   81.1265 0.9017    -88.64 250.8931  False
      AutoCES-N AutoTheta-Multi  100.9267 0.6874  -68.8398 270.6932

#### Final hyperparameter configuration for deployed NHITS and NBEATS Models

In [15]:
df = pd.DataFrame()
for model in ['NHITS','NBEATS']:
    for H in [30,60,90]:
        x = joblib.load(fr'Results COBEQ\{model} (D)\pm10\{H}D_Df.pkl').sort_values(['smape','mae']).reset_index(drop=True)
        df = pd.concat([df, pd.DataFrame(x.iloc[0,:]).T])

df.to_excel(r'x.xlsx')

#### Relative ranking of performance among 13 forecasting models

In [11]:
x = joblib.load(fr'Results COBEQ\Stats (D)\pm10\60D_Df.pkl').sort_values(['smape', 'mae']).reset_index()
x = x.groupby(['pollutant', 'method', 'h', 'freq'])[['mape', 'smape', 'max', 'mae', 'mse']].mean().reset_index().sort_values('smape')
x

,pollutant,method,h,freq,mape,smape,max,mae,mse
12,pm10,NHITS,60,D,56.440165,39.415470,33.668802,10.744117,182.602165
2,pm10,AutoCES-S,60,D,62.820200,41.524825,34.199098,11.364957,201.952443
3,pm10,AutoCES-Z,60,D,66.290413,41.652810,36.014975,11.668088,224.424270
4,pm10,AutoETS-ZAA,60,D,63.798708,41.739622,32.653802,11.531762,211.610373
5,pm10,AutoETS-ZAM,60,D,66.161115,42.065062,32.623492,11.682895,216.496395
11,pm10,NBEATS,60,D,55.242243,43.410193,35.732143,11.407605,212.274200
8,pm10,AutoETS-ZNN,60,D,93.190962,51.586950,39.771450,16.338400,435.871260
10,pm10,AutoTheta-Multi,60,D,120.565750,70.414367,56.668093,22.864375,826.678948
9,pm10,AutoTheta-Add,60,D,125.530810,76.339995,50.685325,23.712332,825.184667
0,pm10,AutoCES-N,60,D,67.598062,97.419377,49.454315,18.424107,521.387412


In [14]:
df = pd.DataFrame()
for model in ['NHITS','NBEATS']:
    for H in [30,60,90]:
        x = joblib.load(fr'Results COBEQ\Stats (D)\pm10\{H}D_Df.pkl').sort_values(['smape','mae']).reset_index()
        x = x.groupby(['pollutant','method','h','freq'])[['mape', 'smape', 'max', 'mae', 'mse']].mean().reset_index().sort_values('smape')
        metrics = ['smape', 'mae', 'mape', 'mse', 'max']
        for metric in metrics:
            x[f'{metric}_rank'] = x[metric].rank(method='min')

        # If you want best = 1, worst = 21, and ranks to be integers
        x[[f'{m}_rank' for m in metrics]] = x[[f'{m}_rank' for m in metrics]].astype(int)
        df = pd.concat([df, x[x['method'] == model]])
        # del df['index']
df.to_excel(r'x.xlsx')

#### Plots

In [ ]:
from statsforecast import StatsForecast
from statsforecast.models import AutoARIMA, AutoCES, AutoETS, AutoTheta

results_stats = []
for pollutant in ['pm10']:
    for h in [365]:

        obj = TimeObject(data, pollutant, agg_freq='D')
        obj.NIXTLA_train_test(split=h)

        season_length = 365 # Monthly data 
        horizon = len(obj.Y_train) # number of predictions

        models = [
            # AutoARIMA(season_length=season_length, alias='AutoARIMA'),
            AutoCES(season_length=season_length, model='Z', alias='AutoCES-Z'),
            AutoCES(season_length=season_length, model='S', alias='AutoCES-S'),
            AutoCES(season_length=season_length, model='P', alias='AutoCES-P'),
            AutoCES(season_length=season_length, model='N', alias='AutoCES-N'),
            AutoTheta(season_length=season_length, decomposition_type="multiplicative", alias='AutoTheta-Multi'),
            AutoTheta(season_length=season_length, decomposition_type="additive", alias='AutoTheta-Add'),
        ]
        models = models + [
            AutoETS(season_length=season_length, model=ets, alias=f'AutoETS-{ets}')
            for ets in [f"{e}{t}{s}" for e in ['Z',]
                        for t in ['Z', 'A', 'N'] 
                        for s in ['Z', 'A', 'M', 'N'] 
        ]]

        frct = StatsForecast(models=models, freq='D')
        frct.fit(df=obj.Y_train)
        predicted = frct.predict(h=h)

        columns = predicted.columns
        columns = columns[(columns != 'ds') & (columns != 'unique_id')]

        results_stats = pd.DataFrame()
        for method in columns:
            obj.plot_forecast(predicted, show_metrics=True, method=method)

In [ ]:
for pollutant in ['pm10']:
    for h in [14, 30, 60, 90]:

        obj = TimeObject(data, pollutant, agg_freq='D')
        obj.NIXTLA_train_test(split=h)
        horizon = len(obj.Y_train) # number of predictions

        # ======================================================================================================

        nbeats = joblib.load(fr"Results COBEQ\NBEATS (W)\{pollutant}\78W_Study.pkl")
        model = NBEATS(
            h=h,
            input_size=nbeats.best_trials[0].params.get('input_size'),
            stack_types=nbeats.best_trials[0].params.get('interpretability')+(nbeats.best_trials[0].params.get('n_stacks')-len(nbeats.best_trials[0].params.get('interpretability')))*['identity'],
            n_blocks=nbeats.best_trials[0].params.get('n_stacks') * [nbeats.best_trials[0].params.get('n_blocks')],
            max_steps=nbeats.best_trials[0].params.get('max_steps'),
            learning_rate=1e-3,
            val_check_steps=10,
        )
        fcst = NeuralForecast(
            models=[model],
            freq='D',
            local_scaler_type=nbeats.best_trials[0].params.get('local_scalar_type')
        )
        fcst.fit(df=obj.Y_train, verbose=False)
        predicted = fcst.predict(df=obj.Y_train, verbose=False)
        obj.plot_forecast(predicted, method='NBEATS', show_metrics=True)

        # ======================================================================================================
        
        nhits = joblib.load(fr"Results COBEQ\NHITS (W)\{pollutant}\78W_Study.pkl")
        model = NHITS(
            h=h,
            input_size=nhits.best_trials[0].params.get('input_size'),
            stack_types=nhits.best_trials[0].params.get('n_stacks')*['identity'],
            n_freq_downsample=nhits.best_trials[0].params.get('n_freq_downsample')+(nhits.best_trials[0].params.get('n_stacks')-len(nhits.best_trials[0].params.get('n_freq_downsample')))*[1],
            n_blocks=nhits.best_trials[0].params.get('n_stacks')*[nhits.best_trials[0].params.get('n_blocks')],
            n_pool_kernel_size=(nhits.best_trials[0].params.get('n_stacks')-len(nhits.best_trials[0].params.get('n_pool_kernel_size')))*[1]+nhits.best_trials[0].params.get('n_pool_kernel_size'),
            pooling_mode="MaxPool1d",
            activation="ReLU",
            interpolation_mode='linear',
            max_steps=nhits.best_trials[0].params.get('max_steps'),
            val_check_steps=10,
        )
        fcst = NeuralForecast(
            models=[model],
            freq='D',
            local_scaler_type=nhits.best_trials[0].params.get('local_scalar_type')
        )
        fcst.fit(df=obj.Y_train, verbose=False)
        predicted = fcst.predict(df=obj.Y_train, verbose=False)
        obj.plot_forecast(predicted, show_metrics=True)